# Exercise 5: Substation Load Optimization

Tasks
1. Normalize the load values for each substation daily
2. Identify hours when load exceeds 85% of its daily peak
3. Reshape the data into (substation, day, hour) form

In [2]:
import numpy as np


# 3 days × 24 hours = 72 time steps

# 4 substations → each has 72 readings

load_data = np.random.randint(40, 150, size=(4, 72)) # shape: (4 substations, 72 hourly readings)
print(load_data)

[[117  86  84 123 136 100 105  85  83 127  96 135 103  50  40 147 112 130
  108  77  44 101  73  73 106 104  98  70  54 103 146  58  78 135  43  74
   65  46 143  70 113  85 110 143 120  96  52  76  50  79  57 126 148  53
  140 127  72 124  40 149 139  61 101  60  70  71  63  44 101  88  97 130]
 [112 128 142  42 114 124 137 149  50 109 137 137  98 116 110  42 129  65
  133 137 103  93 106 127 101 139  98 111  54 135  83 139  49  97  56  44
  138 140 101 146 128 106 132 101  64  64  69 148  46  85  82  81  48 120
   57  89 135  77 138 121 104  85  57 130  47  51  97 143 103  43  81  79]
 [ 74  61  95  41  86  61 129 103  78  66  43  55 102  79 142  91  89  64
   83  82 148 105 133 146  77 142 130  72 126  41  51  53  80  80  63 117
  117  62 125 117  98  99 132  55  91 108  92 133  99  74  74  45  59  85
   50  67  50 102 129 108 143 136 137 130  54  63  41 138  66 137  75  49]
 [126  74  70  91  96 149  97  52 129 129  55  88 125 130  43 111  50  71
   73  53 137  80 143 138  99  43 1

In [12]:
# Task 1
reshaped_data = load_data.reshape((4,3,24), order='C')

In [13]:
# Task 2
max_value = []
min_value = []
n,m,l = reshaped_data.shape
normalised_data = np.zeros(reshaped_data.shape)

for i in reshaped_data:
    maximum = np.max(i)
    max_value.append(maximum)
    minimum = np.min(i)
    min_value.append(minimum)

for i in range(n):
    for j in range(m):
        for k in range(l):
            normalised_data[i][j][k] = (reshaped_data[i][j][k] - min_value[i]) / (max_value[i] - min_value[i])

In [18]:
# Task 3
peak_mask = normalised_data > 0.85

In [19]:
# Task 4
print(load_data.shape)
print(reshaped_data.shape)
print(condition)
print(peak_mask.sum())

(4, 72)
(4, 3, 24)
[[[False False False False  True False False False False False False
    True False False False  True False False False False False False
   False False]
  [False False False False False False  True False False  True False
   False False False  True False False False False  True False False
   False False]
  [False False False False  True False  True False False False False
    True  True False False False False False False False False False
   False False]]

 [[False False  True False False False  True  True False False  True
    True False False False False False False  True  True False False
   False False]
  [False  True False False False  True False  True False False False
   False  True  True False  True False False False False False False
   False  True]
  [False False False False False False False False  True False  True
   False False False False False False False False  True False False
   False False]]

 [[False False False False False False False False Fa

# 1. Grid Stress Anomaly Segmentation
Scenario:
You have voltage data from 8 substations for 60 time steps under transient fault simulation.

- Reshape to (4, 2, 60): 4 zones × 2 substations × time
- Normalize voltage per zone and substation
- Detect time points where voltage dips >15% from local max
- Output boolean mask of anomalies shape: (4, 2, 60)

In [22]:
import numpy as np

np.random.seed(42)
volts = np.random.normal(loc=230, scale=15, size=(8, 60))

In [46]:
# task 1
reshaped_data = volts.reshape((4, 2, 60), order = 'C')

In [47]:
# Task 2
max_value = []
min_value = []
n,m,l = reshaped_data.shape
normalised_data = np.zeros(reshaped_data.shape)

for i in reshaped_data:
    maximum = np.max(i)
    max_value.append(maximum)
    minimum = np.min(i)
    min_value.append(minimum)

for i in range(n):
    for j in range(m):
        for k in range(l):
            normalised_data[i][j][k] = (reshaped_data[i][j][k] - min_value[i]) / (max_value[i] - min_value[i])

In [48]:
# Task 3

local_maximum = []
n,m,l = reshaped_data.shape
for i in range(n):
    for j in range(m):
        local_max = np.max(reshaped_data[i][j])
        local_maximum.append(local_max)

boolean_mask = reshaped_data > (local_maximum[i] * 0.85)
time_points = np.extract(boolean_mask, reshaped_data)

In [49]:
# Task 4

boolean_mask

array([[[False, False, False,  True, False, False,  True, False, False,
         False, False, False, False, False, False, False, False, False,
         False, False,  True, False, False, False, False, False, False,
         False, False, False, False,  True, False, False, False, False,
         False, False, False, False, False, False, False, False, False,
         False, False,  True, False, False, False, False, False, False,
          True, False, False, False, False,  True],
        [False, False, False, False, False,  True, False,  True, False,
         False, False,  True, False,  True, False, False, False, False,
         False, False, False, False,  True, False, False, False, False,
         False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False,
         False,  True, False, False, False, False, False, False,  True,
         False, False, False, False,  True, False]],

       [[False, False,  True, 

# 2. Switched Axis Control Reordering
Scenario:
You simulate power flows across 5 feeders and 6 load points with 48 samples each.

- Move the time axis to the front: → shape (48, 5, 6)
- Then flatten the feeder-load plane → shape: (48, 30)

In [50]:
data = np.random.rand(5, 6, 48) # Shape: (feeder, load_point, time)

In [56]:
new_data = np.moveaxis(data, 2, 0)
new_data.shape

(48, 5, 6)

In [57]:
rehaped_data = new_data.reshape((48, 30), order='C')
rehaped_data.shape

(48, 30)

# 3. Sensor Grid Merging and Slicing

Scenario:
You collect readings from two separate sensor networks (left and right) each of shape (12, 24)—rows × cols.

- Horizontally stack them to shape (12, 48)
- Extract:
    - Top-left quadrant → (6, 24)
    - Bottom-right quadrant → (6, 24)

In [68]:
left = np.random.randint(50, 100, size=(12, 24))
right = np.random.randint(40, 90, size=(12, 24))

In [80]:
horizontal = np.hstack((left, right))
horizontal.shape

(12, 48)

In [79]:
top_left = horizontal[0:6,0:24]
bottom_right = horizontal[6:12,24:48]
print(top_left.shape)
print(bottom_right.shape)

(6, 24)
(6, 24)


# 4. Dynamic Threshold Filtering in 3D

Scenario:
Load measurements across 10 transformers, 3 phases, and 96 time slots.

- For each transformer, compute mean load across all phases
-  Identify time points where load in any phase > 1.15 × mean

In [81]:
load = np.random.normal(loc=100, scale=10, size=(10, 3, 96))

In [88]:
# Task 1
mean_load = np.mean(load, axis=1, keepdims=True)

In [90]:
# Task 2
condition = load > 1.15 * mean_load
identify_time = np.extract(condition, load)

# 5. Rolling 3D Array Transformation

Scenario:
You simulate demand across 12 substations over 72 hours, in 3-hour chunks.

- Create a rolling window view with:
    - Shape = (12, 70, 3) → sliding window of 3 hours
- Then compute the standard deviation across each window per substation

In [2]:
import numpy as np 

substation_data = np.random.randint(80, 180, size=(12, 72))

In [5]:
#Task 1
rolling_window = np.lib.stride_tricks.sliding_window_view(substation_data, window_shape=3, axis=1)

In [6]:
# Task 2
standard_deviation = np.std(rolling_window, axis = 2)
standard_deviation

array([[37.24990679, 34.12070079, 28.54625875, 29.48822741,  7.31816613,
         6.37704216, 26.23398983, 23.62672686,  1.69967317, 18.66071334,
        18.15366507, 24.14309195,  8.04155872, 25.1705295 , 23.32857094,
         8.21921867,  9.67241209,  4.96655481, 11.0855261 , 11.86029792,
        35.05234181, 38.43898484, 35.6121078 , 35.83604641, 33.82635396,
        27.95631513, 12.72792206, 12.08304597, 11.0855261 ,  9.46337971,
         5.88784058,  5.55777733,  6.12825877, 11.4406682 , 20.46134567,
        15.19502843, 20.53181813, 33.2565783 , 34.45125381, 26.71246068,
        20.85398976, 16.00694294, 31.84336666, 38.26225294,  5.73488351,
        11.84154645, 19.60158724, 24.115463  , 11.22497216, 38.42163742,
        30.26916289, 24.16609195, 26.98147513, 22.04540769, 12.08304597,
        35.78019316, 34.98888712, 22.52899366, 26.54974367, 26.73325021,
        28.08320969, 28.77885026, 35.6931366 , 33.67491648, 35.49021774,
        14.05544576,  4.92160769, 41.77186719, 41.5

In [7]:
a =[2]
b = tuple(a)
print(b)

(2,)
